In [1]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import spearmanr
import seaborn as sns
import os

# Paths to data directories
isc_dir = '/Volumes/ARCHIVES/thesis_pipeline/data/ISC_vectors'
adjacency_dir = '/Volumes/ARCHIVES/thesis_pipeline/data/group_level_adjacency_matrices'
hmm_event_bounds_dir = '/Volumes/ARCHIVES/thesis_pipeline/data/group_level_HMM'
figures_dir = '/Volumes/ARCHIVES/thesis_pipeline/figures/HMM_ISRSA_permutations'
os.makedirs(figures_dir, exist_ok=True)

save_data_dir = '/Volumes/ARCHIVES/thesis_pipeline/data/permutation_sig_values'
save_fig_dir = '/Volumes/ARCHIVES/thesis_pipeline/figures/permuation_plots_overlays'
os.makedirs(save_data_dir, exist_ok=True)
os.makedirs(save_fig_dir, exist_ok=True)

# ROI names
roi_names = ['PTL', 'ATL', 'AG', 'IFG', 'MFG', 'IFGorb']

In [ ]:
# Function to normalize a vector to the range [0, 1]
def normalize_vector(vector):
    min_val = np.min(vector)
    max_val = np.max(vector)
    return (vector - min_val) / (max_val - min_val)

# Updated function to visualize ISC and HMM vectors with event boundaries
def plot_normalized_isc_hmm_alignment(isc_vector, hmm_adjacency_vector, event_bounds, roi_name, save_dir, max_trs=500):
    # Normalize the vectors
    isc_vector_norm = normalize_vector(isc_vector[:max_trs])
    hmm_adjacency_vector_norm = normalize_vector(hmm_adjacency_vector[:max_trs])
    event_bounds = [b for b in event_bounds if b < max_trs]  # Limit boundaries to the first max_trs

    # Time points for the plot
    time_points = np.arange(len(isc_vector_norm))
    
    plt.figure(figsize=(12, 6))
    plt.plot(time_points, isc_vector_norm, label="ISC Vector (Normalized)", color="blue", alpha=0.7)
    plt.plot(time_points, hmm_adjacency_vector_norm, label="HMM Adjacency Vector (Normalized)", color="red", alpha=0.7)
    
    # Highlight event boundaries
    for boundary in event_bounds:
        plt.axvline(x=boundary, color="green", linestyle="--", label="Event Boundary" if boundary == event_bounds[0] else "")
    
    # Labels and legend
    plt.title(f"Normalized Temporal Alignment of ISC and HMM Vectors for {roi_name} (First {max_trs} TRs)")
    plt.xlabel("Time Points (TRs)")
    plt.ylabel("Normalized Value")
    plt.legend()
    plt.grid(True)
    plt.tight_layout()
    
    # Save the figure
    save_path = os.path.join(save_dir, f'{roi_name}_Normalized_ISC_HMM_alignment.png')
    plt.savefig(save_path, dpi=300)
    plt.close()
    print(f"Saved normalized plot for {roi_name} at {save_path}")

# Iterate through ROIs and process each with normalization and limited TRs
for roi_name in roi_names:
    isc_path = os.path.join(isc_dir, f'{roi_name}_ISC_vector.npy')
    adjacency_path = os.path.join(adjacency_dir, f'{roi_name}_adjacency_matrix.npy')
    event_bounds_path = os.path.join(hmm_event_bounds_dir, f'{roi_name}_event_bounds.npy')
    
    # Check if all files exist
    if os.path.exists(isc_path) and os.path.exists(adjacency_path) and os.path.exists(event_bounds_path):
        # Load data
        isc_vector = np.load(isc_path)
        hmm_adjacency_matrix = np.load(adjacency_path)
        event_bounds = np.load(event_bounds_path)
        
        # Compute HMM adjacency vector (mean of rows)
        hmm_adjacency_vector = np.mean(hmm_adjacency_matrix, axis=1)
        
        # Generate and save the plot
        plot_normalized_isc_hmm_alignment(isc_vector, hmm_adjacency_vector, event_bounds, roi_name, figures_dir)
    else:
        print(f"Missing data for {roi_name}. Ensure ISC vector, adjacency matrix, and event bounds are available.")

In [ ]:
n_permutations = 5000


# Normalize HMM adjacency vector to [0, 1]
def normalize_vector(vector):
    return (vector - np.min(vector)) / (np.max(vector) - np.min(vector))

# Enhanced function to plot histograms with publication-ready styling
def plot_permutation_histogram(permuted_correlations, observed_correlation, roi_name, perm_type, save_path):
    plt.figure(figsize=(10, 6))
    sns.histplot(permuted_correlations, bins=30, kde=False, color='skyblue', edgecolor='black', label="Permutation Null")
    plt.axvline(observed_correlation, color='red', linestyle='dashed', linewidth=2, label=f'Observed Correlation: {observed_correlation:.4f}')
    plt.axvline(np.percentile(permuted_correlations, 2.5), color='green', linestyle='dashed', linewidth=2, label='95% Significance Thresholds')
    plt.axvline(np.percentile(permuted_correlations, 97.5), color='green', linestyle='dashed', linewidth=2)
    
    # Title and axes
    plt.title(f"Permutation Test ({perm_type.capitalize()}) for {roi_name}", fontsize=16, weight="bold")
    plt.xlabel("Spearman Correlation", fontsize=14, weight="bold")
    plt.ylabel("Frequency", fontsize=14, weight="bold")
    plt.legend(fontsize=12)
    plt.grid(alpha=0.3)
    plt.tight_layout()
    
    # Save the plot
    plt.savefig(save_path, dpi=300)
    plt.close()
    print(f"Saved histogram plot for {roi_name} ({perm_type}) at {save_path}")

def cross_validation_analysis(roi_name, permute_isc=True):
    isc_path = os.path.join(isc_dir, f'{roi_name}_ISC_vector.npy')
    adjacency_path = os.path.join(adjacency_dir, f'{roi_name}_adjacency_matrix.npy')
    
    if not os.path.exists(isc_path) or not os.path.exists(adjacency_path):
        print(f"Data missing for {roi_name}. Skipping analysis.")
        return None
    
    # Load ISC vector and normalize HMM adjacency vector
    isc_vector = np.load(isc_path)
    hmm_adjacency_matrix = np.load(adjacency_path)
    hmm_adjacency_vector = np.mean(hmm_adjacency_matrix, axis=1)
    hmm_adjacency_vector = normalize_vector(hmm_adjacency_vector)
    
    # Compute observed Spearman correlation
    observed_correlation, _ = spearmanr(isc_vector, hmm_adjacency_vector)
    
    # Permutation testing
    permuted_correlations = []
    for _ in range(n_permutations):
        if permute_isc:
            np.random.shuffle(isc_vector)  # Shuffle ISC vector
            permuted_vector = hmm_adjacency_vector  # Keep HMM adjacency vector unchanged
        else:
            permuted_indices = np.random.permutation(hmm_adjacency_matrix.shape[0])
            permuted_matrix = hmm_adjacency_matrix[permuted_indices][:, permuted_indices]
            permuted_vector = np.mean(permuted_matrix, axis=1)
            permuted_vector = normalize_vector(permuted_vector)
        
        # Compute Spearman correlation
        permuted_correlation, _ = spearmanr(isc_vector, permuted_vector)
        permuted_correlations.append(permuted_correlation)
    
    # Save results
    perm_type = "isc_permuted" if permute_isc else "hmm_permuted"
    save_path = os.path.join(save_data_dir, f'{roi_name}_{perm_type}_correlations.npy')
    np.save(save_path, permuted_correlations)
    print(f"Saved {perm_type} correlations for {roi_name} at {save_path}")
    
    # Plot histogram
    plot_path = os.path.join(save_fig_dir, f'{roi_name}_{perm_type}_permutation_test.png')
    plot_permutation_histogram(permuted_correlations, observed_correlation, roi_name, perm_type, plot_path)
# Generate overlay plots
def overlay_distributions(roi_name):
    isc_perm_path = os.path.join(save_data_dir, f'{roi_name}_isc_permuted_correlations.npy')
    hmm_perm_path = os.path.join(save_data_dir, f'{roi_name}_hmm_permuted_correlations.npy')
    
    if not os.path.exists(isc_perm_path) or not os.path.exists(hmm_perm_path):
        print(f"Permutation data missing for {roi_name}. Skipping overlay.")
        return None
    
    isc_permuted_correlations = np.load(isc_perm_path)
    hmm_permuted_correlations = np.load(hmm_perm_path)
    
    # Overlay plot
    plt.figure(figsize=(12, 6))
    bins = np.linspace(-1, 1, 30)
    plt.hist(isc_permuted_correlations, bins=bins, alpha=0.6, label="ISC Permutation Null", color="blue")
    plt.hist(hmm_permuted_correlations, bins=bins, alpha=0.6, label="HMM Permutation Null", color="orange")
    plt.title(f"Overlay of Permutation Null Distributions for {roi_name}", fontsize=16, weight="bold")
    plt.xlabel("Spearman Correlation", fontsize=14, weight="bold")
    plt.ylabel("Frequency", fontsize=14, weight="bold")
    plt.legend(fontsize=12)
    plt.grid(alpha=0.3)
    plt.tight_layout()
    
    # Save the overlay plot
    fig_name = f"{roi_name}_permutation_overlay.png"
    plt.savefig(os.path.join(save_fig_dir, fig_name), dpi=300)
    plt.close()
    print(f"Saved overlay plot for {roi_name} at {os.path.join(save_fig_dir, fig_name)}")

# Run analysis
for roi_name in roi_names:
    cross_validation_analysis(roi_name, permute_isc=True)  # ISC permutation
    cross_validation_analysis(roi_name, permute_isc=False)  # HMM permutation
    overlay_distributions(roi_name)

In [2]:
def overlay_distributions_and_save(roi_name):
    """
    Overlay ISC and HMM permutation null distributions and save the plot.
    """
    # Load ISC and HMM permutation data
    isc_perm_path = os.path.join(save_data_dir, f'{roi_name}_isc_permuted_correlations.npy')
    hmm_perm_path = os.path.join(save_data_dir, f'{roi_name}_hmm_permuted_correlations.npy')
    
    if not os.path.exists(isc_perm_path) or not os.path.exists(hmm_perm_path):
        print(f"Permutation data missing for {roi_name}. Skipping overlay.")
        return
    
    isc_permuted_correlations = np.load(isc_perm_path)
    hmm_permuted_correlations = np.load(hmm_perm_path)
    
    # Automatically determine X-axis range based on data
    min_val = min(isc_permuted_correlations.min(), hmm_permuted_correlations.min())
    max_val = max(isc_permuted_correlations.max(), hmm_permuted_correlations.max())
    bins = np.linspace(min_val, max_val, 30)
    
    # Overlay plot
    plt.figure(figsize=(12, 6))
    plt.hist(isc_permuted_correlations, bins=bins, alpha=0.6, label="ISC Permutation Null", color="blue")
    plt.hist(hmm_permuted_correlations, bins=bins, alpha=0.6, label="HMM Permutation Null", color="orange")
    plt.title(f"Overlay of Permutation Null Distributions for {roi_name}", fontsize=16, weight="bold")
    plt.xlabel("Spearman Correlation", fontsize=14, weight="bold")
    plt.ylabel("Frequency", fontsize=14, weight="bold")
    plt.legend(fontsize=12)
    plt.grid(alpha=0.3)
    plt.tight_layout()
    
    # Save the plot
    save_path = os.path.join(figures_dir, f'{roi_name}_overlay_permutation_null.png')
    plt.savefig(save_path, dpi=300)
    plt.close()
    print(f"Saved overlay plot for {roi_name} at {save_path}")

# Run overlay for all ROIs
for roi_name in roi_names:
    overlay_distributions_and_save(roi_name)


Saved overlay plot for PTL at /Volumes/ARCHIVES/thesis_pipeline/figures/HMM_ISRSA_permutations/PTL_overlay_permutation_null.png
Saved overlay plot for ATL at /Volumes/ARCHIVES/thesis_pipeline/figures/HMM_ISRSA_permutations/ATL_overlay_permutation_null.png
Saved overlay plot for AG at /Volumes/ARCHIVES/thesis_pipeline/figures/HMM_ISRSA_permutations/AG_overlay_permutation_null.png
Saved overlay plot for IFG at /Volumes/ARCHIVES/thesis_pipeline/figures/HMM_ISRSA_permutations/IFG_overlay_permutation_null.png
Saved overlay plot for MFG at /Volumes/ARCHIVES/thesis_pipeline/figures/HMM_ISRSA_permutations/MFG_overlay_permutation_null.png
Saved overlay plot for IFGorb at /Volumes/ARCHIVES/thesis_pipeline/figures/HMM_ISRSA_permutations/IFGorb_overlay_permutation_null.png


In [ ]:
import pandas as pd

def summarize_permutation_results():
    results = []
    
    for roi_name in roi_names:
        isc_perm_path = os.path.join(save_data_dir, f'{roi_name}_isc_permuted_correlations.npy')
        hmm_perm_path = os.path.join(save_data_dir, f'{roi_name}_hmm_permuted_correlations.npy')
        
        if not os.path.exists(isc_perm_path) or not os.path.exists(hmm_perm_path):
            print(f"Permutation data missing for {roi_name}. Skipping summary.")
            continue
        
        # Load permutation data
        isc_permuted_correlations = np.load(isc_perm_path)
        hmm_permuted_correlations = np.load(hmm_perm_path)
        
        # Compute observed correlations
        isc_observed_corr, _ = spearmanr(np.load(os.path.join(isc_dir, f'{roi_name}_ISC_vector.npy')),
                                         normalize_vector(np.mean(np.load(os.path.join(adjacency_dir, f'{roi_name}_adjacency_matrix.npy')), axis=1)))
        
        # ISC Permutations
        isc_mean = np.mean(isc_permuted_correlations)
        isc_std = np.std(isc_permuted_correlations)
        isc_p_value = (np.sum(np.abs(isc_permuted_correlations) >= np.abs(isc_observed_corr)) + 1) / (n_permutations + 1)
        
        # HMM Permutations
        hmm_mean = np.mean(hmm_permuted_correlations)
        hmm_std = np.std(hmm_permuted_correlations)
        hmm_p_value = (np.sum(np.abs(hmm_permuted_correlations) >= np.abs(isc_observed_corr)) + 1) / (n_permutations + 1)
        
        # Append results
        results.append({
            "ROI": roi_name,
            "ISC Observed Corr": isc_observed_corr,
            "ISC Mean Perm Corr": isc_mean,
            "ISC Perm Std": isc_std,
            "ISC P-value": isc_p_value,
            "HMM Mean Perm Corr": hmm_mean,
            "HMM Perm Std": hmm_std,
            "HMM P-value": hmm_p_value
        })
    
    # Convert to DataFrame
    results_df = pd.DataFrame(results)
    results_df = results_df.round(4)  # Round for readability
    
    # Save table as a CSV
    results_csv_path = os.path.join(save_data_dir, "permutation_results_summary.csv")
    results_df.to_csv(results_csv_path, index=False)
    print(f"Saved summary table at {results_csv_path}")
    
    # Display table
    return results_df

# Generate and display the table
summary_table = summarize_permutation_results()
display(summary_table)  # Use Jupyter's built-in display for DataFrame visualization
